<a href="https://colab.research.google.com/github/cybertraining-dsc/fa20-523-301/blob/master/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Project Notebook

The report for this final project can be found at this [link](https://cybertraining-dsc.github.io/report/fa20-523-301/project/project/).

## Part 1 Importing the Functions and Building the Dataframes

This file requires that we import Numpy, Matplotlib, Pylab, Keras, and Pandas

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
import os, sys
import pandas as pd
import io
import requests

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

An upload for the correct *.json file will need to occur. This file is what allows Kaggle to accept our credentials to download the correct datasets.

In [6]:
##import the kaggle.json from local to drive
!pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"hemmerleingavin","key":"a0412c93ecb33476babdf128d77136c9"}'}

At this point we want to make our our own copy of the Kaggle files. This script also runs a test to ensure that it is working appropriately. The test is displayed below.

In [7]:
##make a kaggle and a data folder
#!mkdir ~/.kaggle
#!mkdir data
##copy the kaggle.json to the .kaggle folder then grant permissions
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
#test to see if kaggle is working, should print list of datasets
!kaggle datasets list

ref                                                               title                                                 size  lastUpdated          downloadCount  
----------------------------------------------------------------  --------------------------------------------------  ------  -------------------  -------------  
nehaprabhavalkar/indian-food-101                                  Indian Food 101                                        7KB  2020-09-30 06:23:43           3813  
christianlillelund/donald-trumps-rallies                          Donald Trump's Rallies                               720KB  2020-09-26 10:25:08            987  
heeraldedhia/groceries-dataset                                    Groceries dataset                                    257KB  2020-09-17 04:36:08           4616  
andrewmvd/trip-advisor-hotel-reviews                              Trip Advisor Hotel Reviews                             5MB  2020-09-30 08:31:20           2680  
balraj98/stanford-back

These datasets are the files that the team used for the assignment. The first download is the injuries. This dataset will give a standard for when the injuries occured and the amount of time missed. The other two datasets are various statistics repositories for the players. Each of these are important to build the team's features.

In [8]:
##downloading all the datasets
!kaggle datasets download -d ghopkins/nba-injuries-2010-2018
!kaggle datasets download -d nathanlauga/nba-games
!kaggle datasets download -d pablote/nba-enhanced-stats

  0% 0.00/226k [00:00<?, ?B/s]
100% 226k/226k [00:00<00:00, 67.2MB/s]
 50% 9.00M/18.1M [00:00<00:00, 11.2MB/s]
100% 18.1M/18.1M [00:00<00:00, 20.7MB/s]
 54% 9.00M/16.7M [00:01<00:00, 9.31MB/s]
100% 16.7M/16.7M [00:01<00:00, 15.6MB/s]


In [9]:
##unzipping to the data folder
!unzip nba-injuries-2010-2018.zip -d data
!unzip nba-games.zip -d data
!unzip nba-enhanced-stats.zip -d data

Archive:  nba-injuries-2010-2018.zip
  inflating: data/injuries_2010-2020.csv  
Archive:  nba-games.zip
  inflating: data/games.csv          
  inflating: data/games_details.csv  
  inflating: data/players.csv        
  inflating: data/ranking.csv        
  inflating: data/teams.csv          
Archive:  nba-enhanced-stats.zip
  inflating: data/2012-18_officialBoxScore.csv  
  inflating: data/2012-18_playerBoxScore.csv  
  inflating: data/2012-18_standings.csv  
  inflating: data/2012-18_teamBoxScore.csv  
  inflating: data/2016-17_officialBoxScore.csv  
  inflating: data/2016-17_playerBoxScore.csv  
  inflating: data/2016-17_standings.csv  
  inflating: data/2016-17_teamBoxScore.csv  
  inflating: data/2017-18_officialBoxScore.csv  
  inflating: data/2017-18_playerBoxScore.csv  
  inflating: data/2017-18_standings.csv  
  inflating: data/2017-18_teamBoxScore.csv  
  inflating: data/metadata_officialBoxScore.pdf  
  inflating: data/metadata_playerBoxScore.pdf  
  inflating: data/metadata

Now that the files are downloaded, the CSV's must be created into dataframes. These dataframes will be used to analyze the data and look at varying trends within the sets. 

In [10]:
#create a list for each data set
ds_NBA_Injuries, ds_NBA_Games, ds_NBA_Enhanced = [], [], []

#import csv files as dataframes and save to respective list, injury set first
df_Injuries = pd.read_csv('data/injuries_2010-2020.csv')
df_Injury_Start = df_Injuries[df_Injuries.Acquired.isnull()]
df_Injury_End = df_Injuries[df_Injuries.Relinquished.isnull()]
ds_NBA_Injuries = [df_Injury_Start, df_Injury_End]
#nba games dataset
df_Games_games = pd.read_csv('data/games.csv')
df_Games_gamesDetails = pd.read_csv('data/games_details.csv')
df_Games_players = pd.read_csv('data/players.csv')
df_Games_ranking = pd.read_csv('data/ranking.csv')
df_Games_teams = pd.read_csv('data/teams.csv')
ds_NBA_Games = [df_Games_games, df_Games_gamesDetails, df_Games_players, df_Games_ranking, df_Games_teams]
#nba enhanced stats dataset
df_En_officialBS_1218 = pd.read_csv('data/2012-18_officialBoxScore.csv')
df_En_playerBS_1218 = pd.read_csv('data/2012-18_playerBoxScore.csv')
df_En_standings_1218 = pd.read_csv('data/2012-18_standings.csv')
df_En_teamBS_1218 = pd.read_csv('data/2012-18_teamBoxScore.csv')  
df_En_officialBS_1617 = pd.read_csv('data/2016-17_officialBoxScore.csv')  
df_En_playerBS_1617 = pd.read_csv('data/2016-17_playerBoxScore.csv')
df_En_standings_1617 = pd.read_csv('data/2016-17_standings.csv')
df_En_teamBS_1617 = pd.read_csv('data/2016-17_teamBoxScore.csv')  
df_En_officialBS_1718 = pd.read_csv('data/2017-18_officialBoxScore.csv')  
df_En_playerBS_1718 = pd.read_csv('data/2017-18_playerBoxScore.csv')
df_En_standings_1718 = pd.read_csv('data/2017-18_standings.csv')
df_En_teamBS_1718 = pd.read_csv('data/2017-18_teamBoxScore.csv')  
##data/metadata_officialBoxScore.pdf, data/metadata_playerBoxScore.pdf, data/metadata_standing.pdf, data/metadata_teamBoxScore.pdf  
df_En_teamBS = pd.read_csv('data/teamBoxScore.csv')
ds_NBA_Enhanced = [df_En_officialBS_1218, df_En_officialBS_1617, df_En_officialBS_1718, df_En_playerBS_1218, df_En_playerBS_1617, df_En_playerBS_1718, df_En_standings_1218, df_En_standings_1617, df_En_standings_1718, \
                       df_En_teamBS_1218, df_En_teamBS_1617, df_En_teamBS_1718, df_En_teamBS]


#probably need some more data exploration and some feature engineering

Setting up download function. This script will check to see if the filename has any data, and if it doesn't it will download from the specified url.

In [ ]:
def download(url, filename):
    if len(filename.columns) >0:
        n=0
        #break
    else: 
        #url="https://github.com/cs109/2014_data/blob/master/countries.csv"
        
        s=requests.get(url).content
        filename=pd.read_csv(url, error_bad_lines=False)
        
    return filename

In [ ]:
nBA_injuries = pd.DataFrame()
nBA_games = pd.DataFrame()
nBA_stats = pd.DataFrame()

In [ ]:
print(nBA_injuries)

Empty DataFrame
Columns: []
Index: []


In [ ]:
url='https://www.kaggle.com/ghopkins/nba-injuries-2010-2018?select=injuries_2010-2020.csv'
filename = nBA_injuries
nBA_injuries = download(url, filename)

url = 'https://www.kaggle.com/nathanlauga/nba-games?select=players.csv'
filename = nBA_games
nBA_games = download(url, filename)

url = 'https://www.kaggle.com/pablote/nba-enhanced-stats?select=2012-18_playerBoxScore.csv'
filename = nBA_stats
nBA_stats = download(url, filename)



b'Skipping line 6: expected 1 fields, saw 2\nSkipping line 9: expected 1 fields, saw 3\nSkipping line 10: expected 1 fields, saw 4\nSkipping line 21: expected 1 fields, saw 2\nSkipping line 33: expected 1 fields, saw 8\nSkipping line 42: expected 1 fields, saw 9\nSkipping line 46: expected 1 fields, saw 2\nSkipping line 47: expected 1 fields, saw 6\nSkipping line 52: expected 1 fields, saw 2\nSkipping line 53: expected 1 fields, saw 3\nSkipping line 54: expected 1 fields, saw 2\nSkipping line 55: expected 1 fields, saw 2\nSkipping line 56: expected 1 fields, saw 2\nSkipping line 78: expected 1 fields, saw 49\nSkipping line 88: expected 1 fields, saw 2\nSkipping line 89: expected 1 fields, saw 2\nSkipping line 90: expected 1 fields, saw 2\nSkipping line 91: expected 1 fields, saw 2\nSkipping line 98: expected 1 fields, saw 2\nSkipping line 100: expected 1 fields, saw 2\nSkipping line 108: expected 1 fields, saw 2\nSkipping line 169: expected 1 fields, saw 2\nSkipping line 170: expected 

# Part 2 Building the Keras Model

A link for Keras for us to use can be found [here](https://keras.io/guides/sequential_model/).

In [ ]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)
# Call model on a test input
x = tf.ones((3, 3))
y = model(x)

Insert Layers

In [ ]:
# Create 3 layers
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

# Call layers on a test input
x = tf.ones((3, 3))
y = layer3(layer2(layer1(x)))

# Part 3 Conclusions

This is where the conclusions section will be typed